## Interactive Phi 3 Mini 4K Instruct Chatbot gamit ang Whisper

### Panimula:
Ang Interactive Phi 3 Mini 4K Instruct Chatbot ay isang tool na nagbibigay-daan sa mga user na makipag-ugnayan sa Microsoft Phi 3 Mini 4K instruct demo gamit ang text o audio input. Ang chatbot ay maaaring gamitin para sa iba't ibang gawain, tulad ng pagsasalin, pagkuha ng update sa panahon, at pangkalahatang pagkuha ng impormasyon.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Gumawa ng bagong token  
Magbigay ng bagong pangalan  
Piliin ang write permissions  
Kopyahin ang token at itago ito sa isang ligtas na lugar


Ang sumusunod na Python code ay may dalawang pangunahing gawain: pag-import ng `os` module at pag-set ng isang environment variable.

1. Pag-import ng `os` module:
   - Ang `os` module sa Python ay nagbibigay ng paraan para makipag-ugnayan sa operating system. Pinapayagan nitong magsagawa ng iba't ibang gawain na may kaugnayan sa operating system, tulad ng pag-access sa environment variables, pagtrabaho sa mga files at directories, at iba pa.
   - Sa code na ito, ang `os` module ay ini-import gamit ang `import` statement. Ang statement na ito ay ginagawang available ang mga functionality ng `os` module para magamit sa kasalukuyang Python script.

2. Pag-set ng isang environment variable:
   - Ang environment variable ay isang halaga na maaaring ma-access ng mga program na tumatakbo sa operating system. Isa itong paraan para mag-imbak ng mga configuration settings o iba pang impormasyon na maaaring magamit ng maraming programa.
   - Sa code na ito, isang bagong environment variable ang sine-set gamit ang `os.environ` dictionary. Ang key ng dictionary ay `'HF_TOKEN'`, at ang value ay itinalaga mula sa `HUGGINGFACE_TOKEN` variable.
   - Ang `HUGGINGFACE_TOKEN` variable ay dine-define sa itaas ng code snippet na ito, at ito ay itinalaga ng string value na `"hf_**************"` gamit ang `#@param` syntax. Ang syntax na ito ay madalas ginagamit sa Jupyter notebooks para pahintulutan ang user input at parameter configuration direkta sa notebook interface.
   - Sa pamamagitan ng pag-set ng `'HF_TOKEN'` environment variable, maaari itong ma-access ng ibang bahagi ng programa o ng ibang mga program na tumatakbo sa parehong operating system.

Sa kabuuan, ang code na ito ay nag-i-import ng `os` module at nagse-set ng isang environment variable na pinangalanang `'HF_TOKEN'` gamit ang value na ibinigay sa `HUGGINGFACE_TOKEN` variable.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Ang code snippet na ito ay nagtatakda ng isang function na tinatawag na `clear_output` na ginagamit upang linisin ang output ng kasalukuyang cell sa Jupyter Notebook o IPython. Tingnan natin ang detalye ng code at unawain ang functionality nito:

Ang function na `clear_output` ay tumatanggap ng isang parameter na tinatawag na `wait`, na isang boolean value. Sa default, ang `wait` ay nakatakda sa `False`. Ang parameter na ito ang nagtatakda kung ang function ay maghihintay hanggang may bagong output na magagamit upang palitan ang kasalukuyang output bago ito linisin.

Ang function mismo ay ginagamit upang linisin ang output ng kasalukuyang cell. Sa Jupyter Notebook o IPython, kapag ang isang cell ay naglalabas ng output, tulad ng naka-print na teksto o mga graphical plot, ang output na iyon ay ipinapakita sa ibaba ng cell. Ang `clear_output` function ay nagbibigay-daan sa iyo upang linisin ang output na iyon.

Ang implementasyon ng function ay hindi ibinigay sa code snippet, tulad ng ipinapakita ng ellipsis (...). Ang ellipsis ay kumakatawan sa placeholder para sa aktwal na code na gumaganap ng paglilinis ng output. Ang implementasyon ng function ay maaaring may kinalaman sa pakikipag-ugnayan sa Jupyter Notebook o IPython API upang alisin ang kasalukuyang output mula sa cell.

Sa kabuuan, ang function na ito ay nagbibigay ng isang maginhawang paraan upang linisin ang output ng kasalukuyang cell sa Jupyter Notebook o IPython, na nagpapadali sa pamamahala at pag-update ng ipinapakitang output sa panahon ng interactive na coding sessions.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Gumamit ng text-to-speech (TTS) gamit ang Edge TTS service. Tingnan natin ang mga kaukulang implementasyon ng mga function isa-isa:

1. `calculate_rate_string(input_value)`: Ang function na ito ay tumatanggap ng input value at kinakalkula ang rate string para sa TTS voice. Ang input value ay kumakatawan sa nais na bilis ng pagsasalita, kung saan ang value na 1 ay kumakatawan sa normal na bilis. Kinakalkula ng function ang rate string sa pamamagitan ng pagbabawas ng 1 mula sa input value, pag-multiply nito sa 100, at pagtukoy ng sign base sa kung ang input value ay mas malaki o katumbas ng 1. Ibinabalik ng function ang rate string sa format na "{sign}{rate}".

2. `make_chunks(input_text, language)`: Ang function na ito ay tumatanggap ng input text at isang language bilang mga parameter. Hinahati nito ang input text sa mga chunks batay sa mga panuntunan ng wika. Sa implementasyong ito, kung ang wika ay "English", hinahati ng function ang text sa bawat period (".") at tinatanggal ang anumang leading o trailing whitespace. Pagkatapos, idinadagdag nito ang period sa bawat chunk at ibinabalik ang na-filter na listahan ng mga chunks.

3. `tts_file_name(text)`: Ang function na ito ay bumubuo ng file name para sa TTS audio file batay sa input text. Gumagawa ito ng ilang mga pagbabago sa text: tinatanggal ang trailing period (kung mayroon), kino-convert ang text sa lowercase, tinatanggal ang leading at trailing whitespace, at pinapalitan ang mga space ng underscores. Pagkatapos, pinapaikli nito ang text sa maximum na 25 characters (kung mas mahaba) o ginagamit ang buong text kung ito ay maikli. Sa huli, bumubuo ito ng random na string gamit ang [`uuid`] module at pinagsasama ito sa pinaikling text upang makabuo ng file name sa format na "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ang function na ito ay pinagsasama ang maraming audio files sa isang audio file. Tumatanggap ito ng listahan ng mga audio file paths at isang output path bilang mga parameter. Ang function ay nag-i-initialize ng isang empty `AudioSegment` object na tinatawag na [`merged_audio`]. Pagkatapos, ini-iterate nito ang bawat audio file path, ina-load ang audio file gamit ang `AudioSegment.from_file()` method mula sa `pydub` library, at idinadagdag ang kasalukuyang audio file sa [`merged_audio`] object. Sa huli, ini-export nito ang pinagsamang audio sa tinukoy na output path sa MP3 format.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Ang function na ito ay gumagawa ng TTS operation gamit ang Edge TTS service. Tumatanggap ito ng listahan ng text chunks, bilis ng pagsasalita, pangalan ng voice, at save path bilang mga parameter. Kung ang bilang ng chunks ay higit sa 1, ang function ay gumagawa ng directory para sa pag-store ng mga individual chunk audio files. Pagkatapos, ini-iterate nito ang bawat chunk, binubuo ang Edge TTS command gamit ang `calculate_rate_string()` function, ang pangalan ng voice, at ang chunk text, at ini-execute ang command gamit ang `os.system()` function. Kung matagumpay ang pag-execute ng command, idinadagdag nito ang path ng na-generate na audio file sa isang listahan. Pagkatapos ma-process ang lahat ng chunks, pinagsasama nito ang mga individual audio files gamit ang `merge_audio_files()` function at sine-save ang pinagsamang audio sa tinukoy na save path. Kung isa lang ang chunk, direkta nitong ginagawa ang Edge TTS command at sine-save ang audio sa save path. Sa huli, ibinabalik nito ang save path ng na-generate na audio file.

6. `random_audio_name_generate()`: Ang function na ito ay bumubuo ng random na audio file name gamit ang [`uuid`] module. Bumubuo ito ng random UUID, kino-convert ito sa string, kinukuha ang unang 8 characters, idinadagdag ang ".mp3" extension, at ibinabalik ang random na audio file name.

7. `talk(input_text)`: Ang function na ito ang pangunahing entry point para sa paggawa ng TTS operation. Tumatanggap ito ng input text bilang parameter. Una nitong sinusuri ang haba ng input text upang matukoy kung ito ay isang mahabang pangungusap (mas mahaba o katumbas ng 600 characters). Batay sa haba at sa value ng `translate_text_flag` variable, tinutukoy nito ang wika at bumubuo ng listahan ng text chunks gamit ang `make_chunks()` function. Pagkatapos, bumubuo ito ng save path para sa audio file gamit ang `random_audio_name_generate()` function. Sa huli, tinatawag nito ang `edge_free_tts()` function upang gawin ang TTS operation at ibinabalik ang save path ng na-generate na audio file.

Sa kabuuan, ang mga function na ito ay nagtutulungan upang hatiin ang input text sa mga chunks, bumuo ng file name para sa audio file, gawin ang TTS operation gamit ang Edge TTS service, at pagsamahin ang mga individual audio files sa isang audio file.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Ang implementasyon ng dalawang function: convert_to_text at run_text_prompt, pati na rin ang deklarasyon ng dalawang klase: str at Audio.

Ang function na convert_to_text ay tumatanggap ng audio_path bilang input at isinasalin ang audio sa text gamit ang modelong tinatawag na whisper_model. Una, tinitingnan ng function kung ang gpu flag ay nakatakda sa True. Kung oo, ginagamit ang whisper_model na may ilang partikular na parameter tulad ng word_timestamps=True, fp16=True, language='English', at task='translate'. Kung ang gpu flag ay False, ginagamit ang whisper_model na may fp16=False. Ang resulta ng transcription ay pagkatapos na-save sa isang file na pinangalanang 'scan.txt' at ibinabalik bilang text.

Ang function na run_text_prompt ay tumatanggap ng isang message at isang chat_history bilang input. Ginagamit nito ang phi_demo function upang makabuo ng sagot mula sa chatbot batay sa input na message. Ang nabuo na sagot ay pagkatapos ipinapasa sa talk function, na nagko-convert ng sagot sa isang audio file at ibinabalik ang file path. Ang Audio class ay ginagamit upang ipakita at i-play ang audio file. Ang audio ay ipinapakita gamit ang display function mula sa IPython.display module, at ang Audio object ay nilikha gamit ang autoplay=True parameter, kaya ang audio ay awtomatikong nagsisimula sa pag-play. Ang chat_history ay ina-update gamit ang input na message at ang nabuo na sagot, at isang walang laman na string at ang na-update na chat_history ang ibinabalik.

Ang str class ay isang built-in na klase sa Python na kumakatawan sa isang sequence ng mga character. Nagbibigay ito ng iba't ibang mga method para sa pag-manipula at pag-trabaho sa mga string, tulad ng capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, at iba pa. Ang mga method na ito ay nagbibigay-daan sa iyo na magsagawa ng mga operasyon tulad ng paghahanap, pagpapalit, pag-format, at pag-manipula ng mga string.

Ang Audio class ay isang custom na klase na kumakatawan sa isang audio object. Ginagamit ito upang lumikha ng isang audio player sa Jupyter Notebook environment. Ang klase ay tumatanggap ng iba't ibang mga parameter tulad ng data, filename, url, embed, rate, autoplay, at normalize. Ang data parameter ay maaaring isang numpy array, isang listahan ng mga sample, isang string na kumakatawan sa filename o URL, o raw PCM data. Ang filename parameter ay ginagamit upang tukuyin ang isang lokal na file kung saan kukunin ang audio data, at ang url parameter ay ginagamit upang tukuyin ang URL kung saan ida-download ang audio data. Ang embed parameter ay tumutukoy kung ang audio data ay dapat i-embed gamit ang data URI o i-refer mula sa orihinal na source. Ang rate parameter ay tumutukoy sa sampling rate ng audio data. Ang autoplay parameter ay tumutukoy kung ang audio ay awtomatikong magsisimula sa pag-play. Ang normalize parameter ay tumutukoy kung ang audio data ay dapat i-normalize (i-rescale) sa maximum na posibleng range. Ang Audio class ay nagbibigay din ng mga method tulad ng reload upang i-reload ang audio data mula sa file o URL, at mga attribute tulad ng src_attr, autoplay_attr, at element_id_attr upang makuha ang kaukulang mga attribute para sa audio element sa HTML.

Sa kabuuan, ang mga function at klase na ito ay ginagamit upang isalin ang audio sa text, bumuo ng audio na sagot mula sa chatbot, at ipakita at i-play ang audio sa Jupyter Notebook environment.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
